In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.3682, -0.2422, -0.1509, -0.0962, -0.4735,  0.1804,  0.2164,  0.1559,
         -0.2374, -0.3917, -0.0629,  0.3694,  0.5818,  0.0098,  0.4757, -0.2906,
          0.1022,  0.9775,  0.1083,  0.0229],
        [-0.3813, -0.3567,  0.0536, -0.1175, -0.8355,  0.2874,  0.1175, -0.1463,
         -0.2578, -0.0508, -0.1277,  0.4099,  0.4907,  0.0021,  0.4733, -0.0462,
          0.1421,  0.8043,  0.0551, -0.3775],
        [-0.0933, -0.1595, -0.2106, -0.3381, -0.5380,  0.0966,  0.2252,  0.1043,
         -0.4107, -0.2754, -0.3166,  0.2356,  0.4247, -0.1700,  0.1837,  0.1634,
          0.1296,  0.3178, -0.1123, -0.1454]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1804, 0.2164, 0.1559, 0.0000,
         0.0000, 0.0000, 0.3694, 0.5818, 0.0098, 0.4757, 0.0000, 0.1022, 0.9775,
         0.1083, 0.0229],
        [0.0000, 0.0000, 0.0536, 0.0000, 0.0000, 0.2874, 0.1175, 0.0000, 0.0000,
         0.0000, 0.0000, 0.4099, 0.4907, 0.0021, 0.47

In [13]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [14]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [15]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0160,  0.0087,  0.0216,  ...,  0.0167, -0.0243,  0.0188],
        [-0.0340,  0.0079, -0.0058,  ..., -0.0173,  0.0159,  0.0276]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0260,  0.0210], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0194, -0.0067, -0.0185,  ..., -0.0103, -0.0300, -0.0166],
        [ 0.0187, -0.0416, -0.0229,  ..., -0.0095,  0.0360,  0.0248]],
       device='cuda:0', grad_fn=<Sl